# Binary Brains Team

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [2]:
df = pd.read_csv('Datasets/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [17]:
X_test.shape

(2000, 13)

In [18]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [20]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5026 - accuracy: 0.7962
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4673 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4536 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4408 - accuracy: 0.8054
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4260 - accuracy: 0.8139
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4090 - accuracy: 0.8239
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3941 - accuracy: 0.8351
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3827 - accuracy: 0.8389
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3745 - accuracy: 0.8438
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3674 - accura

250/250 [==============================] - 0s 1ms/step - loss: 0.3376 - accuracy: 0.8627
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3374 - accuracy: 0.8624
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3376 - accuracy: 0.8611
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3383 - accuracy: 0.8608
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3371 - accuracy: 0.8609
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3372 - accuracy: 0.8620
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3368 - accuracy: 0.8625
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3371 - accuracy: 0.8619
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 0.8610
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3365 - accuracy: 

### undersampling
### ================================================

In [21]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [22]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [24]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [25]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 1ms/step - loss: 0.6785 - accuracy: 0.5756
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6566 - accuracy: 0.6171
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6373 - accuracy: 0.6419
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6237 - accuracy: 0.6594
Epoch 5/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6142 - accuracy: 0.6692
Epoch 6/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6059 - accuracy: 0.6741
Epoch 7/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5951 - accuracy: 0.6827
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5870 - accuracy: 0.6907
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5786 - accuracy: 0.7005
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5721 - accura

102/102 [==============================] - 0s 1ms/step - loss: 0.4470 - accuracy: 0.7846
Epoch 82/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4465 - accuracy: 0.7837
Epoch 83/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4450 - accuracy: 0.7852
Epoch 84/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4462 - accuracy: 0.7806
Epoch 85/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.7837
Epoch 86/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4460 - accuracy: 0.7846
Epoch 87/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4454 - accuracy: 0.7824
Epoch 88/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4456 - accuracy: 0.7824
Epoch 89/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4444 - accuracy: 0.7852
Epoch 90/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4449 - accuracy: 

### Oversampling
### ==============================================


In [26]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [27]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [28]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6668 - accuracy: 0.5891
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6130 - accuracy: 0.6765
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5588 - accuracy: 0.7227
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5143 - accuracy: 0.7531
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4967 - accuracy: 0.7571
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4881 - accuracy: 0.7611
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4847 - accuracy: 0.7633
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4812 - accuracy: 0.7622
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4798 - accuracy: 0.7630
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4778 - accura

399/399 [==============================] - 1s 1ms/step - loss: 0.4429 - accuracy: 0.7863
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4416 - accuracy: 0.7856
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4408 - accuracy: 0.7856
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4413 - accuracy: 0.7830
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4415 - accuracy: 0.7848
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4404 - accuracy: 0.7856
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4399 - accuracy: 0.7876
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4394 - accuracy: 0.7878
Epoch 89/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4382 - accuracy: 0.7900
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4395 - accuracy: 

In [29]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [30]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [32]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6542 - accuracy: 0.6137
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6057 - accuracy: 0.6768
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5736 - accuracy: 0.7043
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5434 - accuracy: 0.7286
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5153 - accuracy: 0.7529
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4934 - accuracy: 0.7656
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4813 - accuracy: 0.7717
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4725 - accuracy: 0.7768
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4679 - accuracy: 0.7771
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4636 - accura

399/399 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.8096
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4163 - accuracy: 0.8064
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4149 - accuracy: 0.8085
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4155 - accuracy: 0.8071
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4140 - accuracy: 0.8082
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4142 - accuracy: 0.8097
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4134 - accuracy: 0.8087
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4159 - accuracy: 0.8085
Epoch 89/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4141 - accuracy: 0.8088
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4142 - accuracy: 

### Ensemble 
### ====================================================

In [33]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [34]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [36]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [37]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [38]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [39]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [40]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.7038 - accuracy: 0.5088
Epoch 2/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6732 - accuracy: 0.6134
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6529 - accuracy: 0.6266
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6297 - accuracy: 0.6432
Epoch 5/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6169 - accuracy: 0.6579
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6089 - accuracy: 0.6736
Epoch 7/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6024 - accuracy: 0.6858
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5980 - accuracy: 0.6874
Epoch 9/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5939 - accuracy: 0.6976
Epoch 10/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5902 - accuracy: 0.6963
Epoch 11/

98/98 [==============================] - 0s 1ms/step - loss: 0.4602 - accuracy: 0.7843
Epoch 84/100
98/98 [==============================] - 0s 2ms/step - loss: 0.4592 - accuracy: 0.7827
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4585 - accuracy: 0.7811
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4573 - accuracy: 0.7824
Epoch 87/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4579 - accuracy: 0.7782
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4577 - accuracy: 0.7798
Epoch 89/100
98/98 [==============================] - 0s 2ms/step - loss: 0.4561 - accuracy: 0.7792
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4571 - accuracy: 0.7811
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4593 - accuracy: 0.7792
Epoch 92/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4569 - accuracy: 0.7850
Epoch 93/100


In [41]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6942 - accuracy: 0.5210
Epoch 2/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6699 - accuracy: 0.5949
Epoch 3/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6547 - accuracy: 0.6253
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6418 - accuracy: 0.6285
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6318 - accuracy: 0.6390
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6243 - accuracy: 0.6602
Epoch 7/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6178 - accuracy: 0.6682
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6110 - accuracy: 0.6730
Epoch 9/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6033 - accuracy: 0.6874
Epoch 10/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5934 - accuracy: 0.6963
Epoch 11/

98/98 [==============================] - 0s 1ms/step - loss: 0.4822 - accuracy: 0.7683
Epoch 84/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4808 - accuracy: 0.7696
Epoch 85/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4814 - accuracy: 0.7722
Epoch 86/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4801 - accuracy: 0.7712
Epoch 87/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4824 - accuracy: 0.7686
Epoch 88/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7731
Epoch 89/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4808 - accuracy: 0.7696
Epoch 90/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4806 - accuracy: 0.7677
Epoch 91/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7715
Epoch 92/100
98/98 [==============================] - 0s 1ms/step - loss: 0.4805 - accuracy: 0.7709
Epoch 93/100


In [42]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 1ms/step - loss: 0.6833 - accuracy: 0.6018
Epoch 2/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6398 - accuracy: 0.6339
Epoch 3/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6181 - accuracy: 0.6628
Epoch 4/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6053 - accuracy: 0.6653
Epoch 5/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5963 - accuracy: 0.6801
Epoch 6/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5864 - accuracy: 0.6859
Epoch 7/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5773 - accuracy: 0.6993
Epoch 8/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5712 - accuracy: 0.7054
Epoch 9/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5604 - accuracy: 0.7141
Epoch 10/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.7188
Epoch 11/

87/87 [==============================] - 0s 2ms/step - loss: 0.4577 - accuracy: 0.7827
Epoch 84/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4580 - accuracy: 0.7827
Epoch 85/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4575 - accuracy: 0.7841
Epoch 86/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4576 - accuracy: 0.7816
Epoch 87/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4578 - accuracy: 0.7848
Epoch 88/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4578 - accuracy: 0.7841
Epoch 89/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4582 - accuracy: 0.7787
Epoch 90/100
87/87 [==============================] - 0s 2ms/step - loss: 0.4568 - accuracy: 0.7830
Epoch 91/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4578 - accuracy: 0.7859
Epoch 92/100
87/87 [==============================] - 0s 1ms/step - loss: 0.4557 - accuracy: 0.7856
Epoch 93/100


In [43]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [44]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.73      0.82      1593
           1       0.44      0.82      0.57       407

    accuracy                           0.75      2000
   macro avg       0.69      0.78      0.70      2000
weighted avg       0.84      0.75      0.77      2000

